In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

# Arguments

In [2]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [3]:
pretrained = './log/pggan/128x128/b0/checkpoint.pth.tar'
resume = ''

In [4]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 5000
start_epoch = 0
train_batch = 32
test_batch = 300
lr = 0.01
schedule = [1500, 3000]
momentum = 0
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_style1/l2sp_pggan' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
alpha = 0.1
beta = 0.1
fc_name = 'fc.'

best_acc = 0

In [5]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [6]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot_pggan')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
source_model = EfficientNet.from_name(model_name, num_classes=num_classes)
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    source_model.load_state_dict(torch.load(pretrained)['state_dict'])
    model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128x128/b0/checkpoint.pth.tar'


In [9]:
source_model.to('cuda')
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


In [10]:
for param in source_model.parameters():
    param.requires_grad = False
source_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
source_model_weights = {}
for name, param in source_model.named_parameters():
    source_model_weights[name] = param.detach()

# Model tuning

In [12]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - source_model_weights[name]) ** 2
    return sp_loss

In [13]:
summary(model, input_size=(3,128,128),device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 129, 129]               0
Conv2dStaticSamePadding-2           [-1, 32, 64, 64]             864
         GroupNorm-3           [-1, 32, 64, 64]              64
MemoryEfficientSwish-4           [-1, 32, 64, 64]               0
         ZeroPad2d-5           [-1, 32, 66, 66]               0
Conv2dStaticSamePadding-6           [-1, 32, 64, 64]             288
         GroupNorm-7           [-1, 32, 64, 64]              64
MemoryEfficientSwish-8           [-1, 32, 64, 64]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 32, 1, 1]             288
         I

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc = test(val_target_loader, model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, model, criterion, epoch, use_cuda)

        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        scheduler_warmup.step()

        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)


Epoch: [1 | 5000] LR: 0.010000
1/13 | Total:0:00:02 | ETA:0:00:25 | Loss:0.8547770380973816 | top1:81.25
11/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.8555605032227256 | top1:71.875
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.8756441821654638 | top1:70.83333587646484


/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


26/26 | Total:0:00:17 | ETA:0:00:00 | Loss:0.9644469687571893 | top1:49.0128173828125
107/107 | Total:0:00:37 | ETA:0:00:00 | Loss:0.02826297361508151 | top1:99.70093536376953

Epoch: [2 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:08 | Loss:0.3368217647075653 | top1:78.125
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.520152514631098 | top1:71.59091186523438
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.5144527405500412 | top1:72.13542175292969

Epoch: [3 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.5016435384750366 | top1:75.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.49490847370841284 | top1:71.30682373046875
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.48509934296210605 | top1:72.65625

Epoch: [4 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.48128989338874817 | top1:71.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.4628345045176419 | top1:71.30682373046875
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.45904458810885745 | top1:71.875


1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.1306561529636383 | top1:96.875
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.2678524337031625 | top1:87.21591186523438
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.27434395750363666 | top1:86.97917175292969

Epoch: [32 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.2692887783050537 | top1:84.375
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.3090768104249781 | top1:84.94318389892578
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.3014544794956843 | top1:85.41667175292969

Epoch: [33 | 5000] LR: 0.017000
1/13 | Total:0:00:01 | ETA:0:00:15 | Loss:0.30907413363456726 | top1:78.125
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.2756507112221284 | top1:84.94318389892578
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.27574510996540386 | top1:85.41667175292969

Epoch: [34 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.2890465259552002 | top1:93.75
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.2792562476613305 | top1:87.21591

1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.17665080726146698 | top1:93.75
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.2802052159201015 | top1:88.63636779785156
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.2728761360049248 | top1:88.54167175292969

Epoch: [61 | 5000] LR: 0.024000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.18073801696300507 | top1:90.625
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.1990372741764242 | top1:92.61363983154297
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.19436260064442953 | top1:92.70833587646484

Epoch: [62 | 5000] LR: 0.024000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.11391086876392365 | top1:93.75
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.17762477560476822 | top1:92.32955169677734
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.17573493346571922 | top1:92.44792175292969

Epoch: [63 | 5000] LR: 0.024000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.13665771484375 | top1:96.875
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.15011429177089172 | top1:93.4659

...

11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.24332489615136926 | top1:90.34091186523438
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.23666334276398024 | top1:90.625

Epoch: [467 | 5000] LR: 0.080000
1/13 | Total:0:00:01 | ETA:0:00:14 | Loss:0.11840181052684784 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.1812790808352557 | top1:92.61363983154297
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.17956042662262917 | top1:92.70833587646484

Epoch: [468 | 5000] LR: 0.080000
1/13 | Total:0:00:01 | ETA:0:00:13 | Loss:0.1346646547317505 | top1:93.75
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.23284045674584128 | top1:90.34091186523438
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.22387238591909409 | top1:90.625

Epoch: [469 | 5000] LR: 0.080000
1/13 | Total:0:00:01 | ETA:0:00:17 | Loss:0.2723880708217621 | top1:87.5
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.22885594855655322 | top1:90.625
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.22386248658100763 | top1:91.14583587646484

Epoc

13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.07396305616324146 | top1:97.91667175292969

Epoch: [496 | 5000] LR: 0.080000
1/13 | Total:0:00:01 | ETA:0:00:17 | Loss:0.0967172235250473 | top1:93.75
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.10302382876927202 | top1:96.59091186523438
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.09804630062232415 | top1:96.875

Epoch: [497 | 5000] LR: 0.080000
1/13 | Total:0:00:00 | ETA:0:00:08 | Loss:0.013713568449020386 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.07834761047905142 | top1:96.30682373046875
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.09050239312152068 | top1:95.83333587646484

Epoch: [498 | 5000] LR: 0.080000
1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.04999873414635658 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.036519354209303856 | top1:98.86363983154297
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.03631615871563554 | top1:98.95833587646484

Epoch: [499 | 5000] LR: 0.080000
1/13 | Total:0:00:01 | ETA:0:00:

11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.0633381340991367 | top1:98.29545593261719
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.05994726282854875 | top1:98.4375

Epoch: [525 | 5000] LR: 0.080000
1/13 | Total:0:00:01 | ETA:0:00:13 | Loss:0.021508581936359406 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.06536117737943475 | top1:97.44318389892578
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.06130153344323238 | top1:97.65625

Epoch: [526 | 5000] LR: 0.080000
1/13 | Total:0:00:01 | ETA:0:00:15 | Loss:0.0100514255464077 | top1:100.0
11/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.10783550515770912 | top1:96.30682373046875
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.10666355459640424 | top1:96.35417175292969

Epoch: [527 | 5000] LR: 0.080000
1/13 | Total:0:00:01 | ETA:0:00:13 | Loss:0.06067316606640816 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.06264880299568176 | top1:97.7272720336914
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.059626116727789245 | top1:97.916

1/13 | Total:0:00:01 | ETA:0:00:13 | Loss:0.050836607813835144 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.028935676948590713 | top1:99.1477279663086
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.030116169713437557 | top1:98.95833587646484

Epoch: [554 | 5000] LR: 0.080000
1/13 | Total:0:00:01 | ETA:0:00:14 | Loss:0.02253052033483982 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.008079546927051111 | top1:100.0
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.008572814986109734 | top1:100.0

Epoch: [555 | 5000] LR: 0.080000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.003982208669185638 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.03462881798093969 | top1:99.1477279663086
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.03243139386177063 | top1:99.21875

Epoch: [556 | 5000] LR: 0.080000
1/13 | Total:0:00:01 | ETA:0:00:15 | Loss:0.009607162326574326 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.04963099787181074 | top1:98.01136779785156
13/13 | T

11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.02891796353188428 | top1:99.43182373046875
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.02756049468492468 | top1:99.47917175292969

Epoch: [583 | 5000] LR: 0.080000
1/13 | Total:0:00:01 | ETA:0:00:17 | Loss:0.005747552961111069 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.06756636077030138 | top1:97.7272720336914
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.0680991061963141 | top1:97.65625

Epoch: [584 | 5000] LR: 0.080000
1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.013290254399180412 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.012901094318790869 | top1:100.0
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.012405563921978077 | top1:100.0

Epoch: [585 | 5000] LR: 0.080000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.07748856395483017 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.03809405575421723 | top1:98.86363983154297
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.03582623849312464 | top1:98.95833587646484